In [ ]:
import mne
import numpy as np
from mne.preprocessing import (ICA)
from autoreject import AutoReject
import matplotlib
matplotlib.use("TkAgg")


In [ ]:
eeg_path = "C://Users//neuro//Documents//Git_EEG_Workshop//EEG_Workshop//Data//"  # You will need to change this location
file_name = "VEP_2024-07-05_15-17-52"
file_eeg = eeg_path + file_name + ".eeg"
file_vhdr = eeg_path + file_name + ".vhdr"
file_vmrk = eeg_path + file_name + ".vmrk"

In [ ]:
raw = mne.io.read_raw_brainvision(file_vhdr)
#drop_channels = ['BIP1','BIP2','EOG','TEMP1','ACC1','ACC2','ACC3']
#raw = raw.drop_channels(drop_channels)
#raw.plot()
events_from_annot, event_dict = mne.events_from_annotations(raw)
del event_dict['Stimulus/s5']

In [ ]:
highpass = 1
lowpass = 20
notch = 60

raw_filtered = raw.load_data().filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))
#raw_filtered = raw.resample(resample).filter(highpass, lowpass).notch_filter(np.arange(notch, (notch * 3), notch))

eeg_1020 = raw_filtered.copy().set_eeg_reference(ref_channels = 'average') # ref_channels='['Fz']'
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
eeg_1020 = eeg_1020.set_montage(ten_twenty_montage, on_missing = 'ignore')
#del raw, raw_filtered, ten_twenty_montage
eeg_1020.info['bads'] = []
picks = mne.pick_types(eeg_1020.info, meg=False, eeg=True, stim=False, eog=False, include=[], exclude=[])

epochs = mne.Epochs(eeg_1020,
                    events=events_from_annot,
                    event_id=event_dict,
                    tmin=-0.050,
                    tmax=0.500,   #duration of stimulus or response
                    baseline=None,
                    reject=None,
                    verbose=False,
                    preload=True,
                    detrend=None,
                    event_repeated='drop')


In [ ]:
n_interpolates = np.array([1, 4, 32])
consensus_percs = np.linspace(0, 1.0, 11)
ar = AutoReject(n_interpolates,
                consensus_percs,
                picks=picks,
                thresh_method='random_search',
                random_state=42)    #random n state
epochs_ar = ar.fit_transform(epochs);


In [ ]:
ica = ICA(n_components = 16, max_iter = 'auto', random_state = 123)
ica.fit(epochs_ar)

ica_z_thresh = 1.96
epochs_clean = epochs_ar.copy()
eog_indices, eog_scores = ica.find_bads_eog(epochs_clean,
                                            ch_name=['Fp1', 'F8'],
                                            threshold=ica_z_thresh)
ica.exclude = eog_indices
ica.plot_scores(eog_scores)
ica.plot_sources(epochs_ar)
ica.plot_components()
print(eog_indices)
ica.apply(epochs_clean)
epochs_final = epochs_clean.copy()
#del eeg_1020, epochs, epochs_ar, eog_indices, eog_scores


In [ ]:
baseline_tmin, baseline_tmax = -0.05, 0
baseline = (baseline_tmin, baseline_tmax)

VEP = epochs_final['Stimulus/s1'].apply_baseline(baseline).average()
blank = epochs_final['Stimulus/s2'].apply_baseline(baseline).average()

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz','O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Occipital")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['O1','O2'], combine="mean", show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_O1_O2")

fig = mne.viz.plot_compare_evokeds(VEP, picks=['Oz'], show=False, time_unit="ms")
fig[0].savefig("VEP Data/"+file_name+"-VEP_Oz")

In [ ]:
evokeds = dict(
    Checkerboard=list(epochs_final['Stimulus/s1'].iter_evoked()),
    Blank=list(epochs_final['Stimulus/s2'].iter_evoked()),
)

fig = mne.viz.plot_compare_evokeds(evokeds, 
                                   colors=dict(Checkerboard="red", Blank="black"), 
                                   ci=False, #0.95
                                   picks=['Oz','O1','O2'], time_unit="ms", combine="mean")
fig[0].savefig("VEP Data/"+file_name+"-Compare_Stimuli")